# COIN Project

Authors:

    - Mona
    - Roman
    - Nick
    - Mateo

# 1. Setup

### Packages

In [119]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import tweepy
#import dotenv
import os
import yaml
import datetime
from pprint import pprint
import time
import re


### Environment Variables

In [120]:
#dotenv.load_dotenv()

In [121]:
with open("../twitter.yml", "r") as f:
    config = yaml.safe_load(f)

In [122]:
config

{'accounts': ['Fridays4future',
  'FridayForFuture',
  'GretaThunberg',
  'Luisamneubauer',
  'AnnaUKSCN',
  'AnunaDe',
  'adelaidecharli2',
  'HollyWildChild',
  'AlexandriaV2005',
  'fff_europe',
  'fff_digital',
  'FFF_Sweden',
  'FFF_Scotland',
  'FFF_Berlin',
  'FFFireland',
  'fff_hamburg'],
 'hashtags': {'green': ['EndCoal',
   'EndFossilFuels',
   'PeopleNotProfit',
   'NoMoreEmptyPromises',
   'UprootTheSystem',
   'FridaysForFuture',
   'ClimateAction',
   'ClimateJustice',
   'ClimateEmergency',
   'ClimateStrike',
   'SaveThePlanet'],
  'brown': ['climatescam',
   'climatechangehoax',
   'fakeclimate',
   'climatehoax',
   'globalwarmingisahoax'],
  'neutral': ['ClimateCrisis', 'ClimateChange', 'Climate', 'GlobalWarming']}}

### Database Connection

In [123]:
#TBD

### Twitter API Connection

In [124]:
#Research Access:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAGjtbgEAAAAAyx9oZIai1hXZ9OyDhUUFMZQivpc%3DUunewUXR9hw3nyKQhjqdmfg7zSAoa1nPv6WKLLSPB7OwKwYBP3'

#Initializing the client to request the Twitter API
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

#expansions = ['attachments.poll_ids', 'attachments.media_keys', 'author_id', 'entities.mentions.username', 'geo.place_id', 'in_reply_to_user_id', 'referenced_tweets.id', 'referenced_tweets.id.author_id']
expansions = ['attachments.media_keys','author_id','geo.place_id','referenced_tweets.id', 'entities.mentions.username','referenced_tweets.id.author_id']
max_results = 500
media_fields = ["duration_ms", "height", "media_key", "preview_image_url", "type", "url", "width", "public_metrics", "non_public_metrics", "organic_metrics", "promoted_metrics", "alt_text"]
#next_token = ''#(str | None) – This parameter is used to get the next ‘page’ of results. The value used with the parameter is pulled directly from the response provided by the API, and should not be modified. You can learn more by visiting our page on pagination.
place_fields = ['country', 'country_code', 'full_name', 'geo', 'id', 'name', 'place_type']
#poll_fields #(list[str] | str | None) – poll_fields
#since_id #(int | str | None) – Returns results with a Tweet ID greater than (for example, more recent than) the specified ID. The ID specified is exclusive and responses will not include it. If included with the same request as a start_time parameter, only since_id will be used.
sort_order = 'recency'
start_time = '2022-06-10T00:00:00Z' #(datetime.datetime | str | None) – YYYY-MM-DDTHH:mm:ssZ (ISO 8601/RFC 3339). The oldest UTC timestamp from which the Tweets will be provided. Timestamp is in second granularity and is inclusive (for example, 12:00:01 includes the first second of the minute). By default, a request will return Tweets from up to 30 days ago if you do not include this parameter.
#end_time = '2019-07-15T23:00:00Z'
#tweet_fields = ["attachments", "author_id", "context_annotations", "conversation_id", "created_at", "entities", "geo", "id", "in_reply_to_user_id", "lang", "non_public_metrics", "public_metrics", "organic_metrics", "promoted_metrics", "possibly_sensitive", "referenced_tweets", "reply_settings", "source", "text", "withheld"]
tweet_fields = ["attachments","author_id", "conversation_id","created_at","referenced_tweets","geo","public_metrics"]
#until_id #(int | str | None) – Returns results with a Tweet ID less than (that is, older than) the specified ID. Used with since_id. The ID specified is exclusive and responses will not include it.
#user_fields can be extracted from the .includes attribute of the tweepy response
user_fields = ["created_at", "description", "location", "name", "pinned_tweet_id", "profile_image_url", "protected", "public_metrics", "url", "username", "verified", "withheld"]


# 2. Load and Transform Data

### 2.1 Load

In [150]:
#specify tweets to look up

#query_list = ['from:Luisamneubauer -is:retweet']

query_list = ['FridaysForFuture -is:retweet']


''''query_list = ['from:Fridays4future -is:retweet',
              'from:FridayForFuture -is:retweet',
              'from:GretaThunberg -is:retweet',
              'from:Luisamneubauer -is:retweet',
              'from:AnnaUKSCN -is:retweet',
              'from:AnunaDe -is:retweet',
              'from:adelaidecharli2 -is:retweet',
              'from:HollyWildChild -is:retweet',
              'from:AlexandriaV2005 -is:retweet',
              'from:fff_europe -is:retweet',
              'from:fff_digital -is:retweet',
              'from:FFF_Sweden -is:retweet',
              'from:FFF_Scotland -is:retweet',
              'from:FFF_Berlin -is:retweet',
              'from:FFFireland -is:retweet',
              'from:fff_hamburg -is:retweet'''''

'''query_list = ['FridaysForFuture -is:retweet',
              'SchoolStrike4Climate -is:retweet']'''


"query_list = ['FridaysForFuture -is:retweet',\n              'SchoolStrike4Climate -is:retweet']"

In [151]:
def search_all_tweets(query):
    response = client.search_all_tweets(
                               query=query
                              ,expansions=expansions
                              ,max_results=max_results
                              ,media_fields=media_fields
                              ,place_fields=place_fields
                              ,sort_order=sort_order
                              ,start_time=start_time
                              #,end_time=end_time
                              ,tweet_fields=tweet_fields
                              ,user_fields=user_fields
                            )
    return response

In [152]:
def search_all_tweets_with_next_token(query, next_token):
    response = client.search_all_tweets(
                               query=query
                              ,expansions=expansions
                              ,max_results=max_results
                              ,media_fields=media_fields
                              ,place_fields=place_fields
                              ,sort_order=sort_order
                              ,start_time=start_time
                              #,end_time=end_time
                              ,tweet_fields=tweet_fields
                              ,user_fields=user_fields
                              ,next_token=next_token
                            )
    return response

In [153]:
def fetch_all_data(query, iteration_count, next_token):

    """ Fetches iteration_count * 100 Tweets from Twitter API using the set query params, including the initial next_token
    
    Parameters:
    ------------
    * iteration_count     : Integer indicating the max number of requests 
    * next_token          : String containing the next token for pagination
    
    """
    
    # Initializing the return parameters
    tweets = []
    users = []
    ref_tweets = []
    media = []
    places = []
    
    for i in range(0, iteration_count):

        if(len(next_token) == 0):
            response = search_all_tweets(query)
        else:
            response = search_all_tweets_with_next_token(query, next_token)

        tweets.append(response.data)
        users.append(response.includes['users'])
        ref_tweets.append(response.includes['tweets'])
        if "media" in response.includes.keys():
            media.append(response.includes['media'])
        
        if('places' in response.includes.keys()):
            places.append(response.includes['places'])
            
        try:
            next_token = response.meta['next_token']
        except:
            print('No more Tweets to fetch')
            break


    # Logging the last next_token for pagination into console --> needed for later requesting of the next tweets
    if('next_token' in response.meta):
        print(response.meta['next_token'])
        
    return tweets, users, ref_tweets, media, places

In [154]:
def unpack_dict(unpack_dict):
    unpacked = [v for k, v in unpack_dict.items()]
    return unpacked
def unpack_attachment(attachment_dict):
    return "|".join(attachment_dict["media_keys"]) if "media_keys" in attachment_dict.keys() else np.nan
def unpack_geo(geo_dict):
    return geo_dict["type"], "|".join([str(x) for x in geo_dict["bbox"]])
def unpack_geo_2(geo_dict):
    return geo_dict["place_id"]

In [155]:
def query_tweets(query):    
    tweets, users, ref_tweets, media, places = fetch_all_data(query, 100000, '')

    user_df = pd.DataFrame(data=[user for sublist in users for user in sublist])
    tweet_df = pd.DataFrame(data=[tweet for sublist in tweets for tweet in sublist])
    ref_tweet_df = pd.DataFrame(data=[tweet for sublist in ref_tweets for tweet in sublist])
    media_df = pd.DataFrame(data=[item for sublist in media for item in sublist])
    place_df = pd.DataFrame(data=[place for sublist in places for place in sublist])
    
    user_df[["followers_count", "following_count", "tweet_count", "listed_count"]] = user_df.apply(lambda x: unpack_dict(x["public_metrics"]), axis=1, result_type="expand")
    del user_df["public_metrics"]

    tweet_df[["retweet_count", "reply_count", "like_count", "quote_count"]] = tweet_df.apply(lambda x: unpack_dict(x["public_metrics"]), axis=1, result_type="expand")
    tweet_df["media_keys"] = tweet_df.loc[tweet_df["attachments"].notna()].apply(lambda x: unpack_attachment(x["attachments"]), axis=1, result_type="expand")
    del tweet_df["public_metrics"]
    del tweet_df["attachments"]
    del tweet_df["referenced_tweets"]

    ref_tweet_df[['retweet_count', 'reply_count', 'like_count', 'quote_count']] = ref_tweet_df.apply(lambda x: unpack_dict(x["public_metrics"]), axis=1, result_type="expand")
    ref_tweet_df["media_keys"] = ref_tweet_df.loc[ref_tweet_df["attachments"].notna()].apply(lambda x: unpack_attachment(x["attachments"]), axis=1, result_type="expand")
    #ref_tweet_df['place_id'] = ref_tweet_df.loc[ref_tweet_df["geo"].notna()].apply(lambda x: unpack_geo_2(x["geo"]), axis=1, result_type="expand")
    del ref_tweet_df["public_metrics"]
    del ref_tweet_df["attachments"]
    #del ref_tweet_df["geo"]
    del ref_tweet_df["referenced_tweets"]
    
    '''if "geo" in place_df.columns:
        place_df[["type", "coordinates"]] = place_df.loc[place_df["geo"].notna()].apply(lambda x: unpack_geo(x["geo"]), axis=1, result_type="expand")
        del place_df["geo"]'''
        
    tweet_df.text = tweet_df.text.apply(lambda x: re.sub(r"[^A-Za-z0-9\w\s:@,]", "", x))
    return user_df, tweet_df, ref_tweet_df, place_df, media_df

In [156]:
all_user = all_tweets = all_ref_tweets = all_places = all_media = pd.DataFrame()
for i in query_list:
    print(i)
    user_df, tweet_df, ref_tweet_df, place_df, media_df = query_tweets(i)
    all_user = pd.concat([all_user, user_df])
    all_tweets = pd.concat([all_tweets, tweet_df])
    all_ref_tweets = pd.concat([all_ref_tweets, ref_tweet_df])
    all_places = pd.concat([all_places, place_df])
    all_media = pd.concat([all_media, media_df])
    
    #time.sleep(60)
    

FridaysForFuture -is:retweet
No more Tweets to fetch


In [157]:
#add usernames to all_tweets dataframe
all_user.rename(columns={"id": "author_id"}, inplace=True)
all_tweets = pd.merge(all_tweets,all_user[['author_id','username']],on='author_id', how='inner')
all_tweets = all_tweets.drop_duplicates('id')
all_tweets.shape

(908, 13)

In [158]:
all_user.isnull().sum()

created_at             0
description            0
author_id              0
location             371
name                   0
pinned_tweet_id      595
profile_image_url      0
protected              0
url                    0
username               0
verified               0
followers_count        0
following_count        0
tweet_count            0
listed_count           0
dtype: int64

In [159]:
all_user.head()

,created_at,description,author_id,location,name,pinned_tweet_id,profile_image_url,protected,url,username,verified,followers_count,following_count,tweet_count,listed_count
0,2009-07-21 02:39:13+00:00,Public policy action tank working at the inter...,58678379,New York City,Brighter Green,1.535312e+18,https://pbs.twimg.com/profile_images/147124056...,False,https://t.co/jbv4qJQOiL,BrighterGreenNY,False,1804,1526,9138,130
1,2019-11-26 14:22:07+00:00,netstat -a,1199332435701485569,NaN,SZ40/42,NaN,https://pbs.twimg.com/profile_images/153641497...,False,,42Sz40,False,193,367,173209,9
2,2018-10-20 22:04:14+00:00,"#FridaysForFuture International 🌎 | Global, yo...",1053768884732547072,Earth,Fridays For Future,1.485946e+18,https://pbs.twimg.com/profile_images/148599110...,False,https://t.co/0V5VnFSl7c,Fridays4future,True,137436,1269,9718,1225
3,2013-09-28 23:34:03+00:00,"Ex-Marine, Sozialarbeiter,Hilfe für Obdachlose...",1915629968,Lake of Constance,Der Pep-Flüsterer®️,1.531338e+18,https://pbs.twimg.com/profile_images/153279764...,False,,FCBayernMinga,False,28,32,1892,1
4,2020-03-29 16:58:19+00:00,"80 years ago, 'the struggle for victory over H...",1244306606441070594,UN Website🔗=2015 Declaration→,#MarchForTheUNcharter 14th June 1942 (1st UN Day),1.474175e+18,https://pbs.twimg.com/profile_images/148127986...,False,https://t.co/KjrVWOnQ3d,TheCartHorse1,False,5235,5756,124532,12


In [160]:
all_tweets.head()

,author_id,conversation_id,created_at,entities,geo,id,text,retweet_count,reply_count,like_count,quote_count,media_keys,username
0,58678379,1536418637925388289,2022-06-13 18:42:03+00:00,NaN,NaN,1536418637925388289,Education is a human right FridaysForFuture Ed...,0,0,0,0,3_1536418348635852800,BrighterGreenNY
1,1199332435701485569,1536418394227998723,2022-06-13 18:41:05+00:00,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",NaN,1536418394227998723,@Fridays4future\nTelegram \nwechat \nTwitter \...,0,0,0,0,NaN,42Sz40
2,1199332435701485569,1536417337468932096,2022-06-13 18:36:53+00:00,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",NaN,1536417337468932096,@Fridays4future\nTelegram \nwechat \nTwitter \...,0,0,0,0,NaN,42Sz40
3,1199332435701485569,1536408734301360131,2022-06-13 18:02:42+00:00,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",NaN,1536408734301360131,@Fridays4future\nTelegram \nwechat \nTwitter \...,0,0,0,0,NaN,42Sz40
4,1199332435701485569,1535921031482949632,2022-06-12 09:44:44+00:00,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",NaN,1535921031482949632,@Fridays4future\nTelegram \nwechat \nTwitter \...,0,0,0,0,NaN,42Sz40


In [81]:
[print(x) for x in all_tweets['entities']] 

{'mentions': [{'start': 54, 'end': 70, 'username': 'FridayForFuture', 'id': '1072187272815149057'}]}
{'mentions': [{'start': 0, 'end': 14, 'username': 'TrainTracksEU', 'id': '768547774157783040'}]}
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
{'mentions': [{'start': 132, 'end': 143, 'username': 'derspiegel', 'id': '2834511'}]}
{'mentions': [{'start': 114, 'end': 125, 'username': 'derspiegel', 'id': '2834511'}]}
nan
nan
{'mentions': [{'start': 0, 'end': 12, 'username': 'FFM_Nordend', 'id': '1524378545979641856'}]}
{'mentions': [{'start': 0, 'end': 15, 'username': 'ThomasWendel10', 'id': '1234105220122206212'}, {'start': 16, 'end': 25, 'username': 'MuechNEW', 'id': '1437810368538161158'}]}
{'mentions': [{'start': 0, 'end': 8, 'username': 'zi_news', 'id': '78000347'}]}
{'mentions': [{'start': 0, 'end': 16, 'username': 'reve_rof_repins', 'id': '1513577043912536076'}]}
{'mentions': [{'start': 238, 'end': 249, 'username': 'derspiegel', 'id': '2834511'}]}
{'mentions': [{'start': 0, 'end': 16, 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [31]:
df = pd.DataFrame(columns=[
        'SRC_FOLLOWERS_COUNT', #followers_count
        'SRC_FRIENDS_COUNT', #following_count
        'SRC_LISTED_COUNT', #listed_count
        'SRC_STATUSES_COUNT', #tweet_count
        'SRC_NAME', #username
        'SRC_SCREEN_NAME', #name
        'SRC_ID_STR', #author_id
        'TEXT', #text
        'CREATED_AT', #created_at
        'HASHTAGS', #
        'DST_NAME',
        'DST_SCREEN_NAME',
        'DST_ID_STR',
        'TIME',
        'SRC',
        'DST'])

In [53]:
#Option 1: Save data as separate files

#save dataframe to csv
all_user.to_csv("users.csv", index=False)
all_tweets.to_csv("tweets.csv", index=False)
all_ref_tweets.to_csv("reftweets.csv", index=False, header=False)
all_places.to_csv("places.csv", index=False)
all_media.to_csv("media.csv", index=False)


In [54]:
print('Number of tweets fetched (rows, columns):', all_tweets.shape)
print('Number of users, both authors AND mentioned users (rows, columns):', all_user.shape)
print('Number of ref_tweets (rows, columns):', all_ref_tweets.shape)
print('Number of places, tagged by users in tweets (rows, columns):', all_places.shape)
print('Number of media in tweets (rows, columns):', all_media.shape)

Number of tweets fetched (rows, columns): (892, 12)
Number of users, both authors AND mentioned users (rows, columns): (724, 15)
Number of ref_tweets (rows, columns): (350, 11)
Number of places, tagged by users in tweets (rows, columns): (38, 7)
Number of media in tweets (rows, columns): (390, 9)


In [5]:
'''all_user = pd.read_csv('users.csv', lineterminator='\n')
all_tweets = pd.read_csv('tweets.csv', lineterminator='\n')
 '''

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#Option 2: Save data (user + tweets) in one file

twitter_dataset = pd.merge(all_tweets,all_user, on='author_id')
twitter_dataset = twitter_dataset.drop_duplicates('id')
twitter_dataset.to_csv("tweets_european_accounts.csv", index=False)
print('Shape joint dataset (rows, columns):', twitter_dataset.shape)

In [ ]:
#---user---#
#authors and mentioned users

#---tweets---#
#tweets that are either from a specified account or include a specfified hashtag

#---ref_tweets---#
#A list of Tweets this Tweet refers to. For example, if the parent Tweet is a Retweet,
#a Retweet with comment (also known as Quoted Tweet) or a Reply, it will include the related
#Tweet referenced to by its parent.

#---places---#
#When including the expansions=geo.place_id parameter, this includes a list of referenced places
#in Tweets, if tagged by a user

#---media---#
#When including the expansions=attachments.media_keys parameter, this includes a list of images,
#videos, and GIFs included in Tweets in the form of media objects 